In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d alxmamaev/flowers-recognition
!unzip flowers-recognition.zip

In [0]:
## GETTING DATA READY
import os
import cv2
import numpy as np 

DATA = "/content/flowers"
Categories = ["daisy","dandelion","rose","sunflower","tulip"]

training_data = []
img_size = 128
for item in Categories:
    path  = os.path.join(DATA,item)
    class_num = Categories.index(item)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        try:
            new_array = cv2.resize(img_array,(img_size,img_size))
        except Exception as e:
            pass
        training_data.append([new_array,class_num])

import random
random.shuffle(training_data)
X = []
y = []
for img,label in training_data:
    X.append(img)
    y.append(label)
    
print(len(X))
X_train = np.array(X).reshape(-1,img_size,img_size,3)
print(X_train.shape)


X_train = X_train/255.0
y = np.array(y).reshape(-1,1)
print(y.shape)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_train_hot = encoder.fit_transform(y)
print(y_train_hot.shape)
print(y_train_hot[0:10])

In [4]:
from keras import layers
from keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model,Sequential
from keras.initializers import glorot_uniform
import scipy.misc
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [0]:
def identity_block(X, f, filters, stage, block):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    
    X_shortcut = X
    
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    
   
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

 
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
   
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Filters
    F1, F2, F3 = filters
    
    # For skip coonection
    X_shortcut = X


    
    X = Conv2D(F1, (1, 1), strides = (s,s), padding = 'valid',name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
   

    
    X = Conv2D(F2, (f, f), strides = (1,1), padding='same',name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    
    X = Conv2D(F3, (1, 1), strides = (1,1),padding='valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X =BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

   
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s),padding='valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
   
    
    return X

In [0]:
def ResNet50(input_shape , classes ):
   
    
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

   

    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    
    X = MaxPooling2D(pool_size=(2, 2), name='max_poool')(X) 

    # output layer
    ## Additional dense layer of size 64 
    X = Flatten()(X)
    X = BatchNormalization()(X)
    X = Dense(64, activation='relu', name='fc' + str(64), kernel_initializer = glorot_uniform(seed=0))(X) 
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Creating model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [39]:
model = ResNet50(input_shape = (128, 128, 3), classes = 5)
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,y_train_hot,batch_size = 32,epochs=10,validation_split=0.2)

Train on 3460 samples, validate on 866 samples
Epoch 1/10
3460/3460 [==============================] - 178s 51ms/step - loss: 2.0222 - acc: 0.3142 - val_loss: 5.7714 - val_acc: 0.2263
Epoch 2/10
3460/3460 [==============================] - 45s 13ms/step - loss: 1.3924 - acc: 0.4523 - val_loss: 2.0191 - val_acc: 0.3949
Epoch 3/10
3460/3460 [==============================] - 45s 13ms/step - loss: 1.1598 - acc: 0.5324 - val_loss: 1.6937 - val_acc: 0.3903
Epoch 4/10
3460/3460 [==============================] - 45s 13ms/step - loss: 1.0272 - acc: 0.5902 - val_loss: 1.4009 - val_acc: 0.5393
Epoch 5/10
3460/3460 [==============================] - 44s 13ms/step - loss: 0.9162 - acc: 0.6329 - val_loss: 1.1661 - val_acc: 0.5693
Epoch 6/10
3460/3460 [==============================] - 45s 13ms/step - loss: 0.8110 - acc: 0.6864 - val_loss: 1.1010 - val_acc: 0.5485
Epoch 7/10
3460/3460 [==============================] - 45s 13ms/step - loss: 0.7332 - acc: 0.7171 - val_loss: 1.5845 - val_acc: 0.4781


In [10]:
from keras.applications.resnet50 import ResNet50
model_res50 = Sequential()

model_res50.add( ResNet50(include_top=False, weights='imagenet', input_shape=(128,128,3),pooling='max'))
model_res50.add(BatchNormalization())
model_res50.add(Dense(128, activation='relu'))
model_res50.add(BatchNormalization())
model_res50.add(Dense(64, activation='relu'))
model_res50.add(BatchNormalization())
model_res50.add(Dense(len(Categories), activation='softmax'))

model_res50.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_res50.fit(X_train,y_train_hot,batch_size = 32,epochs=10,validation_split=0.2)


Train on 3460 samples, validate on 866 samples
Epoch 1/10
3460/3460 [==============================] - 65s 19ms/step - loss: 0.7961 - acc: 0.7113 - val_loss: 0.5288 - val_acc: 0.7991
Epoch 2/10
3460/3460 [==============================] - 42s 12ms/step - loss: 0.2521 - acc: 0.9275 - val_loss: 7.0839 - val_acc: 0.3060
Epoch 3/10
3460/3460 [==============================] - 42s 12ms/step - loss: 0.3481 - acc: 0.8766 - val_loss: 0.8400 - val_acc: 0.7102
Epoch 4/10
3460/3460 [==============================] - 42s 12ms/step - loss: 0.1980 - acc: 0.9379 - val_loss: 0.4225 - val_acc: 0.8568
Epoch 5/10
3460/3460 [==============================] - 42s 12ms/step - loss: 0.0966 - acc: 0.9751 - val_loss: 1.5084 - val_acc: 0.4885
Epoch 6/10
3460/3460 [==============================] - 42s 12ms/step - loss: 0.2665 - acc: 0.9133 - val_loss: 1.4379 - val_acc: 0.6016
Epoch 7/10
3460/3460 [==============================] - 42s 12ms/step - loss: 0.1326 - acc: 0.9610 - val_loss: 0.4573 - val_acc: 0.8406
E

In [33]:
from keras.applications.inception_v3 import InceptionV3
model_incp = Sequential()

model_incp.add( InceptionV3(include_top=False, weights='imagenet', input_shape=(128,128,3),pooling='avg'))
model_incp.add(BatchNormalization())
model_incp.add(Dense(128, activation='relu'))
model_incp.add(Dropout(0.3))
model_incp.add(BatchNormalization())
model_incp.add(Dense(64, activation='relu'))
model_incp.add(Dense(len(Categories), activation='softmax'))

model_incp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_incp.fit(X_train,y_train_hot,batch_size = 32,epochs=10,validation_split=0.2)


Train on 3460 samples, validate on 866 samples
Epoch 1/10
3460/3460 [==============================] - 166s 48ms/step - loss: 1.2822 - acc: 0.5014 - val_loss: 1.1183 - val_acc: 0.6247
Epoch 2/10
3460/3460 [==============================] - 36s 10ms/step - loss: 0.7215 - acc: 0.7376 - val_loss: 4.2990 - val_acc: 0.3095
Epoch 3/10
3460/3460 [==============================] - 35s 10ms/step - loss: 0.6863 - acc: 0.7428 - val_loss: 0.9335 - val_acc: 0.6663
Epoch 4/10
3460/3460 [==============================] - 36s 10ms/step - loss: 0.4791 - acc: 0.8303 - val_loss: 0.8117 - val_acc: 0.7252
Epoch 5/10
3460/3460 [==============================] - 35s 10ms/step - loss: 0.3972 - acc: 0.8552 - val_loss: 2.2244 - val_acc: 0.5242
Epoch 6/10
3460/3460 [==============================] - 35s 10ms/step - loss: 0.3999 - acc: 0.8584 - val_loss: 1.4040 - val_acc: 0.6986
Epoch 7/10
3460/3460 [==============================] - 35s 10ms/step - loss: 0.3190 - acc: 0.8873 - val_loss: 1.6022 - val_acc: 0.6224


In [0]:
## TO TEST ANY IMAGE

PATH ="image.jpg"
img = plt.imread(PATH)
from skimage.transform import resize
img =  resize(img,(128,128))
plt.imshow(img)
img = img.reshape(-1,128,128,3)
print(model_res50.predict(img))